In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from typing import Any
from typing import Optional, Union
from pathlib import Path
from bs4 import BeautifulSoup, NavigableString, ResultSet, Tag
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.remote.webdriver import WebDriver
import aiohttp
import asyncio
from aiohttp import ClientError, ClientTimeout
from chompjs import parse_js_object, parse_js_objects
import pprint
import pandas as pd
from pandas import DataFrame
from typing import Callable, Optional, Any
import aiohttp
import asyncio
from sqlalchemy import inspect
from typing import Coroutine, Any
import importlib


def read_from_file(filepath: Path, parser: str) -> BeautifulSoup:
    with filepath.open(mode="r", encoding="utf-8") as file:
        return BeautifulSoup(file.read(), features=parser)


def write_to_file(filepath: Path, soupen: BeautifulSoup):
    with filepath.open(mode="w", encoding="utf-8") as file:
        file.write(str(soupen.prettify()))


def write_string_to_txt_file(file_path, content):
    try:
        # Create a Path object from the provided file path
        path = Path(file_path)

        # Write the content to the file
        with path.open(mode="w", encoding="utf-8") as file:
            file.write(content)

        print(f"Successfully wrote the string to '{file_path}'.")
    except Exception as e:
        print(f"An error occurred: {e}")


def soup(
    source: Union[
        BeautifulSoup, NavigableString, ResultSet, Tag, WebElement, WebDriver, str, Path
    ],
    parser="html.parser",
    filepath: Optional[Path | str] = "temp.html",
) -> BeautifulSoup:
    soupen = None

    if isinstance(source, (BeautifulSoup, NavigableString, ResultSet, Tag)):
        soupen = source
    elif isinstance(source, Path) and source.exists():
        soupen = read_from_file(source, parser)
    elif isinstance(source, str):
        path = Path(source)
        if path.exists():
            soupen = read_from_file(path, parser)
        else:
            soupen = BeautifulSoup(source, features=parser)
    elif isinstance(source, WebElement):
        source_html: str = source.get_attribute("innerHTML")
        soupen = BeautifulSoup(source_html, features=parser)
    elif isinstance(source, WebDriver):
        source_html: str = source.page_source
        soupen = BeautifulSoup(source_html, features=parser)

    if filepath and soupen is not None:
        path = Path(filepath)
        path.parent.mkdir(parents=True, exist_ok=True)
        write_to_file(path, soupen)

    return soupen

In [9]:
from urllib.parse import urlencode

params = [
    ("language-skills[]", ""),
    ("language-skills[]", "de"),
    ("language-skills[]", "en"),
    ("language-skills[]", "fr"),
    ("language-skills[]", "it"),
    ("location", "Switzerland"),
    ("query", "Software Engineer"),
    ("rows", "0")
]

url_params = urlencode(params)
url_params

'language-skills%5B%5D=&language-skills%5B%5D=de&language-skills%5B%5D=en&language-skills%5B%5D=fr&language-skills%5B%5D=it&location=Switzerland&query=Software+Engineer&rows=0'

In [2]:
BASE_URL: str = "https://www.jobs.ch/en/"
BASE_URL_JOBS: str = "https://www.jobs.ch/en/vacancies/"
BASE_URL_COMPANY: str = "https://www.jobs.ch/en/companies/"
BASE_URL_JOB_VAR: str = f"{BASE_URL_JOBS}" + "detail/{0}"
BASE_URL_COMPANY_VAR: str = "https://www.jobs.ch/en/companies/{0}/"
BASE_URL_JOBS_SEARCH: str = (
    BASE_URL_JOBS + "?location={location}&publication-date={pub_date}&term={term}"
)

BASE_API_URL_COMPANY_VAR: str = "https://www.jobs.ch/api/v1/public/company/{company_id}"

In [ ]:
# rows=20
BASE_API_URL_JOBS_VAR = "https://www.jobs.ch/api/v1/public/search?location={location}&query={query}&rows={rows}"
referer="https://www.jobs.ch/en/vacancies/?location=switzerland&term=python"

In [55]:


# Test the function
print(generate_dates(1)) # for 24 hours
    
# generate_dates('24h')
# "2023-08-05%2000%3A00%3A00"
# "2023-08-06%2023%3A59%3A59"



publication-date-from=2023-08-05%2000%3A00%3A00&publication-date-to=2023-08-05%2023%3A59%3A59


In [3]:
from interface.backend import db

await db.init()

2023-08-06 07:32:58 INFO interface.backend.db - init - 26: Db initialized.


In [4]:
async def _make_request(
    session: aiohttp.ClientSession,
    url: str,
    headers: Optional[dict] = None,
    cookies: Optional[dict] = None,
) -> Any:
    async with session.get(url, headers=headers, cookies=cookies) as response:
        if response.status == 200:
            return await response.text()
        print(f"Failed to fetch {url}. Status: {response.status}")
        return None


async def fetch(
    session: aiohttp.ClientSession,
    url: str,
    retries=3,
    timeout_for_wait: Optional[float] = None,
    headers: Optional[dict] = None,
    cookies: Optional[dict] = None,
    function: Optional[Callable] = None,
) -> Any:
    for i in range(retries):
        try:
            content = await asyncio.wait_for(
                _make_request(session, url, headers, cookies), timeout=timeout_for_wait
            )
            return (
                function(content, session, url, headers, cookies)
                if function
                else soup(content)
            )
        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            if i == retries - 1:  # This was the last attempt
                print(f"Failed to fetch {url} after {retries} attempts. Error: {e}")
                return None


async def fetch_all(items, retries=3, timeout_for_session=5) -> list:
    timeout = aiohttp.ClientTimeout(total=timeout_for_session)
    async with aiohttp.ClientSession(timeout=timeout) as session:
        tasks = [
            fetch(
                session,
                item["url"],
                retries,
                timeout_for_session,
                item.get("headers"),
                item.get("cookies"),
                item.get("function"),
            )
            for item in items
        ]
        return await asyncio.gather(*tasks)


In [5]:
def construct_query_url(term="", location="", pub_date="") -> str:
    return BASE_URL_JOBS_SEARCH.format(term=term, location=location, pub_date=pub_date)


async def create_query(term, location, pub_date):
    return await db.create_objs(
        db.models.Query,
        [
            dict(
                url=construct_query_url(term=term, location=location, pub_date=pub_date)
            )
        ],
    )


async def update_query(primary_key, column_value_dict, model):
    return await db.update_objs([primary_key], model, column_value_dict)

In [6]:
async def analyse_html(html: str, filepath: str):
    html_soup: BeautifulSoup = soup(source=html, filepath=filepath)

    script_tag: Tag | NavigableString | None = html_soup.find(
        "script",
        string=lambda x: all(
            text_string in x
            for text_string in [
                "__INIT__",
                "pageTitle",
                "pageDescription",
                "numPages",
                "totalHits",
            ]
        )
        if x is not None
        else False,
    )

    if script_tag:
        data_parsed = [x for x in list(parse_js_objects(script_tag.string)) if x]
        return data_parsed
    else:
        print("Script tag was not found in the html.")


# debug
# write_string_to_txt_file("scrapers/jobsdotch/temp.txt", script_tag.string)


# debug
# write_string_to_txt_file("data_parsed.py", data_parsed)
# with open("scrapers/jobsdotch/data_parsed.py", "w") as file:
# Pretty print the list and write it to the file
# pprint.pprint(data_parsed, stream=file)

### query


In [7]:
async def extract_query_info(job_query: dict) -> dict[str, Any]:
    return {
        "clientClassification": job_query.get("botDetect", {}).get(
            "clientClassification"
        ),
        "pageTitle": job_query.get("seo", {}).get("meta", {}).get("pageTitle"),
        "pageDescription": job_query.get("seo", {})
        .get("meta", {})
        .get("pageDescription"),
        "pageList": str(
            job_query.get("vacancy", {}).get("traversal", {}).get("pageList")
        ),
        "numPages": job_query.get("vacancy", {})
        .get("results", {})
        .get("main", {})
        .get("meta", {})
        .get("numPages"),
        "totalHits": job_query.get("vacancy", {})
        .get("results", {})
        .get("main", {})
        .get("meta", {})
        .get("totalHits"),
        "searchQuery": job_query.get("vacancy", {})
        .get("results", {})
        .get("main", {})
        .get("meta", {})
        .get("searchQuery"),
    }


# queryInfo = extract_query_info(data_parsed[1])
# queryInfo['pageList'] = str(queryInfo['pageList'])
# query_df = pd.DataFrame(query_info).rename_axis('id')
# query_df.index += 1
# query_df
# queryInfo


async def handle_result(pkey, results, model, filepath):
    data_parsed = await analyse_html(results[0], filepath)

    queryInfo = await extract_query_info(data_parsed[1])

    await update_query([pkey], [queryInfo], model)

    return queryInfo, data_parsed

In [7]:
# query_df.to_sql('query', db.engineSync, if_exists='append')


In [ ]:
data_parsed[1]["botDetect"]["clientClassification"]
data_parsed[1]["seo"]["meta"]["pageTitle"]
data_parsed[1]["seo"]["meta"]["pageDescription"]
data_parsed[1]["vacancy"]["traversal"]["pageList"]
data_parsed[1]["vacancy"]["results"]["main"]["meta"]["numPages"]
data_parsed[1]["vacancy"]["results"]["main"]["meta"]["totalHits"]
data_parsed[1]["vacancy"]["results"]["main"]["meta"]["searchQuery"]


'bot'

'585 jobs in Switzerland - jobs.ch'

'Apply now for jobs in Switzerland! We have everything for your next job.'

[1]

30

585

'location=Switzerland'

### subqueries


In [8]:
def generate_subqueries_urls(base_url, numPages) -> list[str]:
    return [f"{base_url}&page={x}" for x in range(1, numPages + 1)]


async def generate_subqueries(queryInfo, queryPrimaryKey):
    numPages = queryInfo["numPages"]
    base_url = (await db.retrieve_objs([queryPrimaryKey], db.models.Query))[0].url
    if numPages > 1:
        urls: list[str] = generate_subqueries_urls(base_url, numPages)
        primaryIds: list[Any] = await db.create_objs(
            db.models.SubQuery, [dict(url=url, queryId=queryPrimaryKey) for url in urls]
        )
    else:
        primaryIds = [queryPrimaryKey]
    return primaryIds

In [9]:
async def mainEn(
    term="",
    location="Switzerland",
    pub_date="",
    headers: Optional[dict] = None,
    cookies: Optional[dict] = None,
) -> Coroutine[Any, Any, None]:
    queryPrimaryKey = (await create_query(term, location, pub_date))[0][0]

    items: list[dict[str, Any]] = [
        {
            "url": (await db.retrieve_objs([queryPrimaryKey], db.models.Query))[0].url,
            "name": "",
            "headers": {},
            "cookies": {},
            "function": None,  # This is a simple example function
        },
        # Add more dictionaries for more URLs
    ]

    results = await fetch_all(items)

    if results:
        queryInfo, data_parsed = await handle_result(
            queryPrimaryKey,
            results,
            db.models.Query,
            f"./data/html/query/{queryPrimaryKey}.html",
        )

        subQueriesPrimaryKeys: list[int] = await generate_subqueries(
            queryInfo, queryPrimaryKey
        )

        # scrape each subQueriesPrimaryKeys
        # fetch each subquery
        # extract subQueryInfo
        # extract Jobs(create jobs table with job+info, queryPm, subqueryPm)
        # increment jobs in in subquery and query

    return queryInfo, data_parsed, subQueriesPrimaryKeys
    # return queryPrimaryKey, queryInfo, subQueriesPrimaryKeys


queryInfo, data_parsed, subQueriesPrimaryKeys = await mainEn()

# queryPrimaryKey, queryInfo, subqueriesPrimaryIds = await mainEn()

In [10]:
subQueries = await db.retrieve_objs(subQueriesPrimaryKeys, db.models.SubQuery)
for idx, subq in enumerate(subQueries, start=1):
    items: list[dict[str, Any]] = [
        {
            "url": subq.url,
            "name": "",
            "headers": {},
            "cookies": {},
            "function": None,  # This is a simple example function
        },
        # Add more dictionaries for more URLs
    ]
    if result := await fetch_all(items):
        queryInfo: dict[str, Any] = await handle_result(
            subq.id,
            result,
            db.models.SubQuery,
            f"./data/html/subqueries/{subq.queryId}/{subq.id}.html",
        )

    await asyncio.sleep(1)
    if idx == 2:
        break

In [15]:
JOB_LISTING_COLS: list[str] = [
    "title",
    "id",
    "initialPublicationDate",
    "publicationDate",
    "employmentGrades",
    "preview",
    "logo",
    "isActive",
    "languageSkills",
    "place",
    "salaryFrom",
    "salaryTo",
    "company.name",
    "company.id",
    "company.slug",
    "company.visible",
    "company.logoImage.src",
    "company.logoImage.height",
    "company.logoImage.width",
    "company_url",
    "job_url",
]


def construnct_job_url(job_id) -> str:
    return BASE_URL_JOB_VAR.format(job_id)


def construnct_company_url(company_id) -> str:
    return BASE_URL_COMPANY_VAR.format(company_id)

def extract_job_info(job) -> dict[str, Any]:
    return {
        "title": job.get("title", ""),
        "id": job.get("id", ""),
        "initialPublicationDate": job.get("initialPublicationDate", ""),
        "publicationDate": job.get("publicationDate", ""),
        "employmentGrades": job.get("employmentGrades", ""),
        "preview": job.get("preview", ""),
        "logo": job.get("logo", ""),
        "isActive": job.get("isActive", ""),
        "languageSkills": job.get("languageSkills", ""),
        "place": job.get("place", ""),
        "salaryFrom": job.get("salaryFrom", ""),
        "salaryTo": job.get("salaryTo", ""),
        "company_name": job.get("company.name", ""),
        "company_id": job.get("company.id", ""),
        "company_slug": job.get("company.slug", ""),
        "company_visible": job.get("company.visible", ""),
        "company_logoImage_src": job.get("company.logoImage.src", ""),
        "company_logoImage_height": job.get("company.logoImage.height", ""),
        "company_logoImage_width": job.get("company.logoImage.width", ""),
        "company_url": construnct_company_url(job.get("company.id", "")),
        "job_url": construnct_job_url(job.get("id", "")),
    }

async def scrape_jobs(data_parsed):
    jobs_df: DataFrame = pd.json_normalize(
        data_parsed[1]["vacancy"]["results"]["main"]["results"]
    )
    jobs_df.loc[:, "company_url"] = jobs_df["company.slug"].agg(construnct_company_url)
    jobs_df.loc[:, "job_url"] = jobs_df["id"].agg(construnct_job_url)
    jobs_df.loc[:, "languageSkills"] = jobs_df.loc[:, "languageSkills"].astype(str)
    
    jobs_df['initialPublicationDate'] = pd.to_datetime(jobs_df['initialPublicationDate'], format='%Y-%m-%dT%H:%M:%S%z')
    jobs_df['publicationDate'] = pd.to_datetime(jobs_df['publicationDate'], format='%Y-%m-%dT%H:%M:%S%z')
    
    
    jobs: DataFrame = jobs_df.loc[:, JOB_LISTING_COLS].set_index("id")
    

    job_dicts = [extract_job_info(job[1]) for job in  jobs.iterrows()]
    return await db.create_objs(db.models.Job, job_dicts)

await scrape_jobs(data_parsed)

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: jobs.id
[SQL: INSERT INTO jobs (id, title, "initialPublicationDate", "publicationDate", "employmentGrades", preview, logo, "isActive", place, "salaryFrom", "salaryTo", "languageSkills", "company.name", "company.id", "company.slug", "company.visible", "company.logoImage.src", "company.logoImage.height", "company.logoImage.width", "company.url", job_url, "queryId", "subQueryId", "dateLog") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP)]
[parameters: (('', 'Head of Finance - Banking', '2023-08-04 17:06:49.000000', '2023-08-04 17:05:11.000000', '100%', 'Client: \nA leading, innovative, modern bank with a strong presence in Switzerland. We have been asked to find someone with CFO potential for this ...', nan, 1, '1211 Geneva, Switzerland', None, None, "[{'language': 'fr', 'level': 3}, {'language': 'en', 'level': 3}]", 'FINDERS SA', '69679', '69679-finders-sa', 1, nan, nan, nan

In [ ]:
'2023-08-04T17:06:49+02:00'

### jobs


In [ ]:
data_parsed[1]["vacancy"]["results"]["main"]["results"]
# write_string_to_txt_file("data_parsed.py", data_parsed)
with open("jobs.py", "w") as file:
    # Pretty print the list and write it to the file
    pprint.pprint(_, stream=file)


In [15]:
jobs_df: DataFrame = pd.json_normalize(
    data_parsed[1]["vacancy"]["results"]["main"]["results"]
)
jobs_df.columns


Index(['applicationQuestions', 'benefits', 'categories', 'contactAddress',
       'contactDetails', 'contactPerson', 'contacts', 'employmentGrades',
       'employmentGradesList', 'employmentPositionIds', 'employmentTypeIds',
       'headhunterApplicationAllowed', 'id', 'initialPublicationDate',
       'isActive', 'isInsecureExternal', 'isWebVacancy', 'languageSkills',
       'layoutType', 'listingTags', 'locations', 'offerId', 'place', 'preview',
       'publicationDate', 'regions', 'salaryFrom', 'salaryTo',
       'sourceHostname', 'title', 'videos', 'regionId', 'displayType',
       'company.id', 'company.name', 'company.slug', 'company.visible', 'logo',
       'company.companyLogoFile', 'company.logoImage.src',
       'company.logoImage.height', 'company.logoImage.width', 'mood'],
      dtype='object')

In [25]:
job_listing_cols: list[str] = [
    "title",
    "id",
    "initialPublicationDate",
    "publicationDate",
    "employmentGrades",
    "preview",
    "logo",
    "isActive",
    "languageSkills",
    "place",
    "salaryFrom",
    "salaryTo",
    "company.name",
    "company.id",
    "company.slug",
    "company.visible",
    "company.logoImage.src",
    "company.logoImage.height",
    "company.logoImage.width",
    "company_url",
    "job_url",
]


def construnct_job_url(job_id) -> str:
    return BASE_URL_JOB_VAR.format(job_id)


def construnct_company_url(company_id) -> str:
    return BASE_URL_COMPANY_VAR.format(company_id)


jobs_df.loc[:, "company_url"] = jobs_df["company.slug"].agg(construnct_company_url)
jobs_df.loc[:, "job_url"] = jobs_df["id"].agg(construnct_job_url)
jobs_df.loc[:, "languageSkills"] = jobs_df.loc[:, "languageSkills"].astype(str)

In [ ]:
jobs: DataFrame = jobs_df.loc[:, job_listing_cols].set_index("id")
jobs.shape

(20, 20)

In [ ]:
# save the dataframe to a table named 'users'
jobs.drop(["languageSkills"], axis=1).to_sql("jobs2", engineSync, if_exists="append")

20

### job


In [23]:
def extract_job_info(job) -> dict[str, Any]:
    return {
        "title": job.get("title", ""),
        "id": job.get("id", ""),
        "initialPublicationDate": job.get("initialPublicationDate", ""),
        "publicationDate": job.get("publicationDate", ""),
        "employmentGrades": job.get("employmentGrades", ""),
        "preview": job.get("preview", ""),
        "logo": job.get("logo", ""),
        "isActive": job.get("isActive", ""),
        "languageSkills": job.get("languageSkills", ""),
        "place": job.get("place", ""),
        "salaryFrom": job.get("salaryFrom", ""),
        "salaryTo": job.get("salaryTo", ""),
        "company.name": job.get("company.name", ""),
        "company.id": job.get("company.id", ""),
        "company.slug": job.get("company.slug", ""),
        "company.visible": job.get("company.visible", ""),
        "company.logoImage.src": job.get("company.logoImage.src", ""),
        "company.logoImage.height": job.get("company.logoImage.height", ""),
        "company.logoImage.width": job.get("company.logoImage.width", ""),
        "company.url": construnct_company_url(job.get("company.id", "")),
        "url": construnct_job_url(job.get("id", "")),
    }


# extract_job_info(jobs_df.iloc[0])

In [ ]:
jobs_df.iloc[0]["title"]
jobs_df.iloc[0]["id"]
jobs_df.iloc[0]["initialPublicationDate"]
jobs_df.iloc[0]["publicationDate"]
jobs_df.iloc[0]["employmentGrades"]
jobs_df.iloc[0]["preview"]
jobs_df.iloc[0]["logo"]
jobs_df.iloc[0]["isActive"]
jobs_df.iloc[0]["languageSkills"]
jobs_df.iloc[0]["place"]
jobs_df.iloc[0]["salaryFrom"]
jobs_df.iloc[0]["salaryTo"]
jobs_df.iloc[0]["company.name"]
jobs_df.iloc[0]["company.id"]
jobs_df.iloc[0]["company.slug"]
jobs_df.iloc[0]["company.visible"]
jobs_df.iloc[0]["company.logoImage.src"]
jobs_df.iloc[0]["company.logoImage.height"]
jobs_df.iloc[0]["company.logoImage.width"]


'Summer Camps Junior Coaches - Various Activities'

'c031309d-69c4-4efc-bcc9-e66b8f3b80d3'

'2023-07-04T10:06:55+02:00'

'2023-08-03T01:58:57+02:00'

'80% – 100%'

...

'78430-ecolint'

True

nan

nan

nan